# 본문에서는 soynlp를 사용
#### https://datascienceschool.net/view-notebook/31eaecec4187428a8dfcab5f686bda8b/ soynlp 설명 사이트 
#### 단어를 cohenson 위주로 설명
#### word_score["연합"].right_branching_entropy => 0.42721236711742844
#### word_score["연합뉴스"].right_branching_entropy => 3.8967810761022053
#### word_score["연합뉴스는"].right_branching_entropy => 0.410116318288409
#### 이런식으로 워드 스코어를 만들어준다. 이를 통해 komoran의 단어사전을 만들어 줄 수 있다.


In [4]:
import konlpy
from konlpy.tag import Komoran
from pprint import pprint
from collections import defaultdict
from collections import Counter
komoran=Komoran()
print(konlpy.__version__)


import pandas as pd

0.5.2


In [177]:
import datetime

year = 2020 #2020
month = 7 #7
day = 25 #28
lenth = 56 #26
# The size of each step in days
day_delta = datetime.timedelta(days=1)

start_date = datetime.date(year, month, day) + 1*day_delta # 가장 첫번째 day는 concat을 위한 base corpus 로 설정하기 위해 빼준다. 
end_date = start_date + lenth*day_delta # 숫자를 설정하여 일수를 조절 할 수 있다. 

date_list = []
for i in range((end_date - start_date).days):
    date_list.append(str(start_date + i*day_delta))
   # print(date_list)
print(start_date)
print(end_date)

2020-07-26
2020-09-20


In [178]:
corpus_start_date = start_date - 1*day_delta  # 위에서 
Data=pd.read_excel(f"D:\Turnaround_Project\YG_DATA\YG_DATAyg {corpus_start_date}.xlsx") 
corpus_start=Data['title'] 

#!pip install xlrd
for i in date_list :
    Data=pd.read_excel(f'D:\Turnaround_Project\YG_DATA\YG_DATAyg {i}.xlsx') ##데이터 가져오기 
    #corpus.append(Data['title'])  ##데이터 text 지정 - 열이 여러개일때
    corpus = Data['title']
    corpus_start = pd.concat([corpus_start, corpus])
    #print(corpus_start)
corpus = corpus_start
print(corpus)

0                        나도 ‘따상’ 꿀맛 좀 보자…개미들 공모주에 취했다
1                                          블랙핑크 세계1위!
2           '유희열의 스케치북' 이하이, 다이어트 후 달라진 몸매…'시청자도 놀랐다'
3     이하이 “YG→AOMG 이적, 유희열 안테나 연락 없더라”(스케치북)[결정적장면...
4                    한승연, 여고생이라고 해도 믿을수 있는 최강의 동안 비주얼
                           ...                       
70             블랙핑크, 美 유명 패션지 커버! 음악·패션 넘나드는 아이콘 '주목'
71                  블랙핑크X셀레나고메즈 'Ice Cream' 리릭 비디오 공개
72                       BTS의 분신이 광고도 찍는다...아이돌의 아바타들
73               블랙핑크 리사, 독보적인 원피스 소화력…사랑스러운 뿔머리 '눈길'
74                          안병용 의정부시장 “청년체감형 정책 적극개발”
Name: title, Length: 5239, dtype: object


In [179]:
from soynlp.word import WordExtractor
word_extractor=WordExtractor(max_left_length=10, max_right_length=6, min_frequency=5)
word_extractor.train(corpus)
scores=word_extractor.word_scores()


from soynlp.noun import LRNounExtractor_v2
noun_extractor=LRNounExtractor_v2()
nouns=noun_extractor.train_extract(corpus,min_noun_frequency=2) #적어도 2번 이상 나온 단어 
print(len(nouns)) #추출된 단어의 갯수 확인 
#nouns['국립암센터'] #특정단어의 빈도수와 단어점수를 알고 싶을때 사용 



high_frequency=[(word, *score) for word, score in nouns.items() 
                if not word.isdigit() and len(word)>1]
sorted(high_frequency, key=lambda x:(-x[1], x[0]))[:1000]
# 단어   (빈도수, cohesion)

training was done. used memory 1.106 Gbry 1.106 Gb
all cohesion probabilities was computed. # words = 4900
all branching entropies was computed # words = 9610
all accessor variety was computed # words = 9610
[Noun Extractor] use default predictors
[Noun Extractor] num features: pos=3929, neg=2321, common=107
[Noun Extractor] counting eojeols
[EojeolCounter] n eojeol = 12688 from 5239 sents. mem=1.106 Gb                    
[Noun Extractor] complete eojeol counter -> lr graph
[Noun Extractor] has been trained. #eojeols=40942, mem=1.122 Gb
[Noun Extractor] batch prediction was completed for 2461 words
[Noun Extractor] checked compounds. discovered 111 compounds
[Noun Extractor] postprocessing detaching_features : 728 -> 728
[Noun Extractor] postprocessing ignore_features : 728 -> 709
[Noun Extractor] postprocessing ignore_NJ : 709 -> 709
[Noun Extractor] 709 nouns (111 compounds) with min frequency=2
[Noun Extractor] flushing was done. mem=1.130 Gb                    
[Noun Extractor] 51

[('블랙핑크', 798, 1.0),
 ('YG', 566, 0.75),
 ('공개', 377, 1.0),
 ('신곡', 315, 0.6666666666666666),
 ('양현석', 307, 1.0),
 ('데뷔', 275, 1.0),
 ('트레저', 274, 1.0),
 ('신인', 234, 1.0),
 ('고메즈', 203, 1.0),
 ('뮤비', 202, 1.0),
 ('포스터', 199, 0.6),
 ('MV', 196, 0.75),
 ('돌파', 193, 1.0),
 ("Cream'", 168, 1.0),
 ('컴백', 153, 1.0),
 ('K팝', 149, 1.0),
 ('앨범', 144, 1.0),
 ("'아이스크림'", 142, 1.0),
 ('혐의', 139, 1.0),
 ('1위', 136, 1.0),
 ('최초', 133, 1.0),
 ('대표', 111, 1.0),
 ("'BOY'", 105, 1.0),
 ('영상', 103, 0.6666666666666666),
 ('세계', 98, 1.0),
 ('뮤직비디오', 97, 1.0),
 ('발매', 94, 1.0),
 ("'사랑해'", 91, 1.0),
 ('그룹', 87, 0.5),
 ('매력', 84, 1.0),
 ("That'", 81, 1.0),
 ('정규앨범', 81, 1.0),
 ('글로', 79, 0.9186046511627907),
 ('예고', 77, 1.0),
 ('싱글', 76, 1.0),
 ('8월', 73, 1.0),
 ('Cream’', 73, 1.0),
 ('5억뷰', 72, 1.0),
 ('걸그룹', 71, 1.0),
 ('안무', 71, 1.0),
 ('출연', 66, 1.0),
 ("'YG", 64, 1.0),
 ('10월', 64, 1.0),
 ('BTS', 64, 1.0),
 ('기록', 64, 0.3333333333333333),
 ('빅히트', 63, 1.0),
 ('선주문', 63, 1.0),
 ('출석', 63, 1.0),
 ('9월', 62

In [180]:
import re
RE_FILTER = re.compile("=[.,!?\"':;~()]…")

sort_hf = sorted(high_frequency, key=lambda x:(-x[1], x[0]))[:1000]
word = []
for i in range(len(sort_hf)):
    print(re.sub(RE_FILTER, "", str(sort_hf[i][0])))


블랙핑크
YG
공개
신곡
양현석
데뷔
트레저
신인
고메즈
뮤비
포스터
MV
돌파
Cream'
컴백
K팝
앨범
'아이스크림'
혐의
1위
최초
대표
'BOY'
영상
세계
뮤직비디오
발매
'사랑해'
그룹
매력
That'
정규앨범
글로
예고
싱글
8월
Cream’
5억뷰
걸그룹
안무
출연
'YG
10월
BTS
기록
빅히트
선주문
출석
9월
상장
넷플릭스
솔로
'뚜두뚜두'
아티스트
제니
차트
발표
확정
모두
도박
법원
인정
4위
데뷔곡
스포티파이
최고
기대
'K팝
신기록
신인'
아이돌
가수
다큐
스트리밍
음원
SM
시장
정규
무대
제목
함께
근황
등극
위너
"블랙핑크
'설강화'
비디오
아바타
"YG
3차례
달콤
전세계
참여
IPO
시작
타이틀곡
해외
콘테스트
‘뚜두뚜두’
강렬
라이브
여성
주인공
7일
방예담
보석함
콘셉트
특별
3위
4년
사랑
주가
'트레저'
12人
댓'
러브'
비주얼
캐릭터
조회수
가득
광복절
상승
에미넴
통화
6일
That’
관심
도전
방탄소년단
보이그룹
열풍
팬들
‘사랑해’
기대감
모델
싱글앨범
올해
질문
'솔로'
26시간
4400만명
묵묵부답
벌써
성장
안무영상
영화
준비
판매량
피처링
행보
활동
히로카즈
41시간
네이버
상큼
주목
9일
가온차트
소유
시런
싹쓸이
언택트
코로나19
탄생
한국
현장
M/V
공모주
데뷔앨범
변신
선물
여부
역대급
예상
이하이
참석
코로나
팬사인회
표정
"데뷔
‘K팝
가을
경신
달성
멤버
올라
음악방송
접수
폭풍성장
하루
'글로
다른
생일
시간
시너지
시총
오픈
음원차트
퍼포먼스
13위
YG엔터
‘설강화’
그래미
완주
입증
제작
질문에
청약
2020년
돌아
러블리
산다라박
씨엘
예승이
오늘(18일)
우리
직진
캐스팅
투자
하락
'스타로드'
4주년
CJCGV
`아이스크림`
관광홍보대사
광고
김도훈
대회
러브’
로맨틱
이번
일본
취재진
'7번방
'보이'
1000억
5주년
7년
AOMG
감독
관련주
논란
등장
모습
사진
상승세
소통
엔터株
여름
역사
영향력
완성
이상
인정…취재진
조언
지원
청량
최단기
활약
'취재진
FNC

In [181]:
#위의 결과를 바탕으로 사용자 사전 만들기
##사용자 사전 추가하기 & 사용자사전이 잘 되는지 테스트
##사용자 사전은 메모장 형태로 되어있다. 단어 tab 형태 로 쓰면 사용자 단어에 추가되는 방식

sent='생일 선물 여부 예상 특별 강렬 벌써 소유 역대급 오픈'
print(f'코모란 형태소 분리 : {komoran.nouns(sent)}')
##사용자 사전 추가하기 & 사용자사전이 잘 되는지 테스트
komoran_userdic=Komoran(userdic='./userdicCanSur.txt')
# 형태소분석 결과 보기 komoran.pos(sent)
print(f'사용자 정의사전 추가 : {komoran_userdic.nouns(sent)}')

코모란 형태소 분리 : ['생일', '선물', '여부', '예상', '소유', '역대', '급', '오픈']
사용자 정의사전 추가 : ['생일', '선물', '여부', '예상', '특별', '강렬', '벌써', '소유', '역대급', '오픈']
